In [ ]:
# 安装固定版本的transformers库（重要：XTTS v2不支持>=4.50.0版本）
!pip install "transformers<4.50.0"

# 安装TTS引擎和相关依赖
!pip install --upgrade TTS

# 安装Web框架和API服务
!pip install fastapi uvicorn

# 安装网络代理工具
!pip install pyngrok

# 安装深度学习框架
!pip install torch

# 安装音频处理库
!pip install pydub

# 安装飞书SDK和HTTP请求库
!pip install requests

# 安装FFmpeg（音频格式转换必需）
!apt-get update && apt-get install -y ffmpeg


In [ ]:
# 导入必要的库
import torch
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import XttsAudioConfig, XttsArgs
from TTS.config.shared_configs import BaseDatasetConfig

# 将XTTS相关配置类添加到PyTorch安全加载白名单
# 这是为了解决PyTorch 2.6+版本的序列化安全限制
torch.serialization.add_safe_globals([
    XttsConfig,           # XTTS模型配置
    XttsAudioConfig,      # XTTS音频配置
    BaseDatasetConfig,    # 基础数据集配置
    XttsArgs             # XTTS参数配置
])

print("✅ PyTorch兼容性修复完成")
print(f"当前PyTorch版本: {torch.__version__}")


In [ ]:
# 配置ngrok认证token
# 请将下面的YOUR_NGROK_TOKEN替换为您在ngrok.com获取的实际token
NGROK_TOKEN = "YOUR_NGROK_TOKEN"  # 🔑 请在这里填入您的ngrok token

# 验证token格式（基本检查）
if NGROK_TOKEN == "YOUR_NGROK_TOKEN" or len(NGROK_TOKEN) < 20:
    print("❌ 请先设置有效的ngrok token!")
    print("📖 获取步骤：")
    print("   1. 访问 https://ngrok.com 注册账户")
    print("   2. 登录后在Dashboard页面找到Authtoken")
    print("   3. 复制token并替换上面的YOUR_NGROK_TOKEN")
else:
    # 配置ngrok认证
    import subprocess
    result = subprocess.run(['ngrok', 'config', 'add-authtoken', NGROK_TOKEN], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ ngrok认证配置成功")
    else:
        print(f"❌ ngrok配置失败: {result.stderr}")


In [ ]:
# 上传土耳其语音频样本
from google.colab import files
import os

print("📤 请上传您的土耳其语音频样本文件...")
print("💡 建议文件名：turkish_speaker.wav")
print("⏱️  建议时长：5-15秒")
print("🎯 要求：清晰的土耳其语发音，无背景噪音")

# 文件上传界面
uploaded = files.upload()

# 获取上传的文件名
if uploaded:
    speaker_filename = list(uploaded.keys())[0]
    print(f"✅ 音频样本上传成功: {speaker_filename}")
    print(f"📁 文件大小: {len(uploaded[speaker_filename]) / 1024:.1f} KB")
    
    # 检查文件是否存在
    if os.path.exists(speaker_filename):
        print(f"✅ 文件验证成功: {speaker_filename}")
        SPEAKER_WAV_PATH = speaker_filename
    else:
        print("❌ 文件上传失败，请重试")
else:
    print("❌ 未检测到上传文件")
    # 使用默认样本（如果有的话）
    SPEAKER_WAV_PATH = None


In [ ]:
# 导入TTS相关库
from TTS.api import TTS
import torch

# 检查GPU可用性
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️  计算设备: {device}")

if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🎮 GPU型号: {gpu_name}")
    print(f"💾 GPU内存: {gpu_memory:.1f} GB")
else:
    print("⚠️  警告：未检测到GPU，将使用CPU（速度较慢）")

print("\n📥 正在下载XTTS v2模型...")
print("⏱️  首次下载需要几分钟时间，请耐心等待")
print("📦 模型大小约1.8GB")


In [ ]:
# 初始化XTTS v2模型
try:
    # 创建TTS实例
    # model_name: 指定使用XTTS v2多语言模型
    # progress_bar: 是否显示进度条
    # gpu: 是否使用GPU加速
    tts = TTS(
        model_name="tts_models/multilingual/multi-dataset/xtts_v2",
        progress_bar=True,
        gpu=(device == "cuda")
    )
    
    print("✅ XTTS v2模型加载成功！")
    print(f"🌍 支持的语言: {', '.join(tts.languages) if hasattr(tts, 'languages') else '多种语言'}")
    
except Exception as e:
    print(f"❌ 模型加载失败: {e}")
    print("🔧 可能的解决方案：")
    print("   1. 检查网络连接")
    print("   2. 重启Colab运行时")
    print("   3. 确认transformers版本正确")
    raise


In [ ]:
# 飞书API配置
# 请在飞书开放平台 https://open.feishu.cn 创建应用并获取以下信息
FEISHU_APP_ID = "YOUR_FEISHU_APP_ID"           # 应用标识
FEISHU_APP_SECRET = "YOUR_FEISHU_APP_SECRET"   # 应用密钥
FEISHU_VERIFICATION_TOKEN = "YOUR_VERIFICATION_TOKEN"  # 事件验证Token
FEISHU_ENCRYPT_KEY = ""  # 事件加密密钥（可选，如果启用了加密则填写）

# 飞书API基础URL
FEISHU_API_BASE = "https://open.feishu.cn/open-apis"

# 验证飞书配置
if (FEISHU_APP_ID == "YOUR_FEISHU_APP_ID" or 
    FEISHU_APP_SECRET == "YOUR_FEISHU_APP_SECRET" or 
    FEISHU_VERIFICATION_TOKEN == "YOUR_VERIFICATION_TOKEN"):
    print("❌ 请先配置飞书机器人信息!")
    print("📖 配置步骤：")
    print("   1. 访问 https://open.feishu.cn 登录飞书开放平台")
    print("   2. 创建企业自建应用")
    print("   3. 获取App ID和App Secret")
    print("   4. 配置机器人权限和事件订阅")
    print("   5. 获取Verification Token")
    print("   6. 将这些信息填入上面的变量中")
else:
    print("✅ 飞书机器人配置已设置")
    print(f"📱 App ID: {FEISHU_APP_ID[:8]}...")
    
print("\n💡 飞书机器人权限要求：")
print("   - 接收消息 v2")
print("   - 发送消息")
print("   - 获取用户基本信息")


In [ ]:
# 导入FastAPI相关库
from fastapi import FastAPI, Request, HTTPException, Header
from fastapi.responses import FileResponse, JSONResponse
from pydub import AudioSegment
import uvicorn
import threading
import os
import traceback
import hashlib
import hmac
import json
import base64
import requests
from datetime import datetime
from typing import Optional

# 创建FastAPI应用实例
app = FastAPI(
    title="飞书 Turkish TTS API",
    description="将土耳其语文本转换为MP3语音的API服务，专为飞书机器人优化",
    version="1.0.0"
)

# 全局变量：保存TTS实例和音频样本路径
TTS_MODEL = tts
SPEAKER_SAMPLE = SPEAKER_WAV_PATH

print("🚀 FastAPI应用创建成功")
print(f"📁 使用的音频样本: {SPEAKER_SAMPLE}")

# 飞书访问令牌缓存
feishu_access_token = None
feishu_token_expire_time = 0


In [ ]:
# 飞书API辅助函数

def get_feishu_access_token():
    """获取飞书访问令牌"""
    global feishu_access_token, feishu_token_expire_time
    
    current_time = datetime.now().timestamp()
    
    # 如果token还未过期，直接返回
    if feishu_access_token and current_time < feishu_token_expire_time:
        return feishu_access_token
    
    # 请求新的access_token
    url = f"{FEISHU_API_BASE}/auth/v3/tenant_access_token/internal"
    headers = {"Content-Type": "application/json; charset=utf-8"}
    data = {
        "app_id": FEISHU_APP_ID,
        "app_secret": FEISHU_APP_SECRET
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        result = response.json()
        
        if result.get("code") == 0:
            feishu_access_token = result["tenant_access_token"]
            feishu_token_expire_time = current_time + result["expire"] - 300  # 提前5分钟过期
            print("✅ 飞书访问令牌获取成功")
            return feishu_access_token
        else:
            print(f"❌ 获取飞书访问令牌失败: {result}")
            return None
    except Exception as e:
        print(f"❌ 飞书API调用异常: {e}")
        return None

def verify_feishu_request(timestamp: str, nonce: str, signature: str, body: bytes) -> bool:
    """验证飞书请求签名"""
    if not FEISHU_VERIFICATION_TOKEN:
        return True  # 如果没有配置验证token，跳过验证
    
    # 构造验证字符串
    string_to_sign = f"{timestamp}{nonce}{FEISHU_VERIFICATION_TOKEN}{body.decode('utf-8')}"
    
    # 计算签名
    calculated_signature = hashlib.sha256(string_to_sign.encode('utf-8')).hexdigest()
    
    return calculated_signature == signature

def send_feishu_message(chat_id: str, message_type: str, content: dict):
    """发送飞书消息"""
    access_token = get_feishu_access_token()
    if not access_token:
        return False
    
    url = f"{FEISHU_API_BASE}/im/v1/messages"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json; charset=utf-8"
    }
    
    data = {
        "receive_id": chat_id,
        "msg_type": message_type,
        "content": json.dumps(content, ensure_ascii=False)
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        result = response.json()
        return result.get("code") == 0
    except Exception as e:
        print(f"❌ 发送飞书消息失败: {e}")
        return False

def upload_feishu_file(file_path: str, file_type: str = "mp3"):
    """上传文件到飞书"""
    access_token = get_feishu_access_token()
    if not access_token:
        return None
    
    url = f"{FEISHU_API_BASE}/im/v1/files"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    try:
        with open(file_path, 'rb') as f:
            files = {
                'file': (os.path.basename(file_path), f, f'audio/{file_type}')
            }
            data = {
                'file_type': 'mp3',
                'file_name': os.path.basename(file_path)
            }
            
            response = requests.post(url, headers=headers, files=files, data=data)
            result = response.json()
            
            if result.get("code") == 0:
                return result["data"]["file_key"]
            else:
                print(f"❌ 上传文件失败: {result}")
                return None
    except Exception as e:
        print(f"❌ 文件上传异常: {e}")
        return None

print("✅ 飞书API辅助函数定义完成")


In [ ]:
# 定义API路由

@app.get("/")
async def root():
    """根路径：API状态检查"""
    return {
        "message": "飞书 Turkish TTS API",
        "status": "running",
        "timestamp": datetime.now().isoformat(),
        "version": "1.0.0",
        "platform": "feishu",
        "endpoints": {
            "tts": "POST /tts - 文本转语音",
            "health": "GET /health - 健康检查",
            "feishu_webhook": "POST /feishu/events - 飞书事件接收"
        }
    }

@app.get("/health")
async def health_check():
    """健康检查端点"""
    try:
        # 检查TTS模型是否可用
        model_status = "available" if TTS_MODEL else "unavailable"
        speaker_status = "available" if SPEAKER_SAMPLE and os.path.exists(SPEAKER_SAMPLE) else "unavailable"
        
        # 检查飞书API连接
        feishu_status = "available" if get_feishu_access_token() else "unavailable"
        
        return {
            "status": "healthy",
            "tts_model": model_status,
            "speaker_sample": speaker_status,
            "feishu_api": feishu_status,
            "timestamp": datetime.now().isoformat()
        }
    except Exception as e:
        return JSONResponse(
            status_code=500,
            content={"status": "unhealthy", "error": str(e)}
        )

@app.post("/tts")
async def create_tts(request: Request):
    """
    文本转语音API端点
    
    接收JSON格式的文本，返回MP3格式的语音文件
    """
    try:
        # 解析请求数据
        data = await request.json()
        text = data.get("text", "")
        language = data.get("language", "tr")  # 默认土耳其语
        user_id = data.get("user_id", "unknown")  # 用户ID（用于日志）
        
        # 输入验证
        if not text.strip():
            raise HTTPException(status_code=400, detail="文本内容不能为空")
        
        if len(text) > 1000:
            raise HTTPException(status_code=400, detail="文本长度不能超过1000字符")
        
        print(f"🎯 收到TTS请求 (用户: {user_id}): {text[:50]}...") if len(text) > 50 else print(f"🎯 收到TTS请求 (用户: {user_id}): {text}")
        print(f"🌍 语言: {language}")
        
        # 生成唯一的输出文件名
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        wav_filename = f"output_{timestamp}.wav"
        mp3_filename = f"output_{timestamp}.mp3"
        
        # 生成语音
        if SPEAKER_SAMPLE and os.path.exists(SPEAKER_SAMPLE):
            # 使用声音克隆
            TTS_MODEL.tts_to_file(
                text=text,
                file_path=wav_filename,
                speaker_wav=SPEAKER_SAMPLE,
                language=language
            )
            print("✅ 语音生成完成（使用声音克隆）")
        else:
            # 使用默认说话人
            TTS_MODEL.tts_to_file(
                text=text,
                file_path=wav_filename,
                language=language
            )
            print("✅ 语音生成完成（使用默认说话人）")
        
        # 转换为MP3格式
        if os.path.exists(wav_filename):
            sound = AudioSegment.from_wav(wav_filename)
            sound.export(mp3_filename, format="mp3")
            
            # 清理临时WAV文件
            os.remove(wav_filename)
            
            print(f"🎵 MP3文件生成成功: {mp3_filename}")
            
            # 返回MP3文件
            return FileResponse(
                mp3_filename,
                media_type="audio/mpeg",
                filename=f"tts_{timestamp}.mp3"
            )
        else:
            raise HTTPException(status_code=500, detail="语音文件生成失败")
            
    except HTTPException:
        # 重新抛出HTTP异常
        raise
    except Exception as e:
        # 捕获其他异常
        error_msg = f"TTS处理错误: {str(e)}"
        print(f"❌ {error_msg}")
        print(f"🔍 详细错误: {traceback.format_exc()}")
        raise HTTPException(status_code=500, detail=error_msg)

print("✅ API路由定义完成")


In [ ]:
# 飞书事件处理路由

@app.post("/feishu/events")
async def handle_feishu_events(
    request: Request,
    x_lark_request_timestamp: Optional[str] = Header(None),
    x_lark_request_nonce: Optional[str] = Header(None),
    x_lark_signature: Optional[str] = Header(None)
):
    """
    处理飞书事件回调
    
    接收飞书发送的各种事件，包括消息事件
    """
    try:
        # 获取请求体
        body = await request.body()
        
        # 验证请求签名（如果配置了验证token）
        if FEISHU_VERIFICATION_TOKEN and x_lark_signature:
            if not verify_feishu_request(
                x_lark_request_timestamp or "",
                x_lark_request_nonce or "",
                x_lark_signature,
                body
            ):
                print("❌ 飞书请求签名验证失败")
                raise HTTPException(status_code=401, detail="签名验证失败")
        
        # 解析事件数据
        try:
            event_data = json.loads(body.decode('utf-8'))
        except json.JSONDecodeError:
            print("❌ 飞书事件数据解析失败")
            raise HTTPException(status_code=400, detail="JSON格式错误")
        
        # 处理URL验证（首次配置webhook时）
        if event_data.get("type") == "url_verification":
            challenge = event_data.get("challenge", "")
            print(f"🔍 飞书URL验证请求: {challenge}")
            return {"challenge": challenge}
        
        # 处理消息事件
        if event_data.get("header", {}).get("event_type") == "im.message.receive_v1":
            return await handle_message_event(event_data)
        
        # 其他事件类型
        print(f"📝 收到飞书事件: {event_data.get('header', {}).get('event_type', 'unknown')}")
        return {"code": 0, "msg": "success"}
        
    except HTTPException:
        raise
    except Exception as e:
        print(f"❌ 飞书事件处理异常: {e}")
        print(f"🔍 详细错误: {traceback.format_exc()}")
        return JSONResponse(
            status_code=500,
            content={"code": -1, "msg": str(e)}
        )

async def handle_message_event(event_data: dict):
    """处理飞书消息事件"""
    try:
        # 提取事件信息
        event = event_data.get("event", {})
        sender = event.get("sender", {})
        message = event.get("message", {})
        
        # 获取消息信息
        user_id = sender.get("sender_id", {}).get("user_id", "")
        chat_id = message.get("chat_id", "")
        message_type = message.get("message_type", "")
        message_id = message.get("message_id", "")
        
        print(f"📨 收到飞书消息 (用户: {user_id}, 聊天: {chat_id})")
        
        # 只处理文本消息
        if message_type != "text":
            print(f"⚠️  暂不支持消息类型: {message_type}")
            return {"code": 0, "msg": "success"}
        
        # 解析消息内容
        try:
            content = json.loads(message.get("content", "{}"))
            text = content.get("text", "").strip()
        except json.JSONDecodeError:
            print("❌ 消息内容解析失败")
            return {"code": 0, "msg": "success"}
        
        if not text:
            print("⚠️  空消息，跳过处理")
            return {"code": 0, "msg": "success"}
        
        print(f"💬 消息内容: {text}")
        
        # 检查是否为命令
        if text.startswith("/"):
            return await handle_command(chat_id, text, user_id)
        
        # 生成语音
        await generate_and_send_voice(chat_id, text, user_id)
        
        return {"code": 0, "msg": "success"}
        
    except Exception as e:
        print(f"❌ 消息处理异常: {e}")
        return {"code": 0, "msg": "success"}  # 返回成功，避免飞书重复推送

async def handle_command(chat_id: str, command: str, user_id: str):
    """处理命令消息"""
    if command == "/help" or command == "/帮助":
        help_text = """🤖 飞书Turkish TTS机器人使用说明：

📝 **基本用法**：
• 直接发送土耳其语文本，我会转换为语音回复
• 支持中文、英文等多种语言（主要优化土耳其语）

🎯 **命令列表**：
• /help - 显示帮助信息
• /status - 查看系统状态
• /test - 测试TTS功能

💡 **使用提示**：
• 文本长度建议控制在500字符以内
• 为获得最佳效果，请使用标准土耳其语
• 语音生成可能需要15-30秒时间

🔧 **技术支持**：
• 基于开源XTTS v2模型
• 运行在Google Colab免费GPU上
• 支持声音克隆和多语言TTS"""
        
        send_feishu_message(chat_id, "text", {"text": help_text})
        
    elif command == "/status" or command == "/状态":
        # 检查系统状态
        model_status = "✅ 正常" if TTS_MODEL else "❌ 异常"
        speaker_status = "✅ 已加载" if SPEAKER_SAMPLE and os.path.exists(SPEAKER_SAMPLE) else "⚠️  未加载"
        api_status = "✅ 正常" if get_feishu_access_token() else "❌ 异常"
        
        status_text = f"""📊 系统状态报告：

🤖 **TTS模型**: {model_status}
🎤 **音频样本**: {speaker_status}  
🔗 **飞书API**: {api_status}
⏰ **运行时间**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

💾 **性能信息**:
• GPU: {'可用' if torch.cuda.is_available() else '不可用'}
• 模型: XTTS v2 多语言TTS
• 支持语言: 土耳其语、英语等"""
        
        send_feishu_message(chat_id, "text", {"text": status_text})
        
    elif command == "/test" or command == "/测试":
        test_text = "Merhaba! Bu bir test mesajıdır. TTS sistemi çalışıyor."
        await generate_and_send_voice(chat_id, test_text, user_id, is_test=True)
        
    else:
        send_feishu_message(chat_id, "text", {"text": f"❓ 未知命令: {command}\n发送 /help 查看可用命令"})
    
    return {"code": 0, "msg": "success"}

async def generate_and_send_voice(chat_id: str, text: str, user_id: str, is_test: bool = False):
    """生成语音并发送到飞书"""
    try:
        # 发送处理中提示
        if is_test:
            send_feishu_message(chat_id, "text", {"text": "🧪 正在进行TTS测试，请稍候..."})
        else:
            send_feishu_message(chat_id, "text", {"text": "🎵 正在生成语音，请稍候..."})
        
        # 生成唯一文件名
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        wav_filename = f"feishu_output_{timestamp}.wav"
        mp3_filename = f"feishu_output_{timestamp}.mp3"
        
        # 生成语音
        if SPEAKER_SAMPLE and os.path.exists(SPEAKER_SAMPLE):
            TTS_MODEL.tts_to_file(
                text=text,
                file_path=wav_filename,
                speaker_wav=SPEAKER_SAMPLE,
                language="tr"
            )
        else:
            TTS_MODEL.tts_to_file(
                text=text,
                file_path=wav_filename,
                language="tr"
            )
        
        # 转换为MP3
        if os.path.exists(wav_filename):
            sound = AudioSegment.from_wav(wav_filename)
            sound.export(mp3_filename, format="mp3")
            os.remove(wav_filename)  # 清理临时文件
            
            # 上传到飞书
            file_key = upload_feishu_file(mp3_filename)
            
            if file_key:
                # 发送语音消息
                audio_content = {"file_key": file_key}
                success = send_feishu_message(chat_id, "audio", audio_content)
                
                if success:
                    print(f"✅ 语音消息发送成功 (用户: {user_id})")
                else:
                    send_feishu_message(chat_id, "text", {"text": "❌ 语音发送失败，请稍后重试"})
            else:
                send_feishu_message(chat_id, "text", {"text": "❌ 语音文件上传失败，请稍后重试"})
            
            # 清理本地文件
            if os.path.exists(mp3_filename):
                os.remove(mp3_filename)
                
        else:
            send_feishu_message(chat_id, "text", {"text": "❌ 语音生成失败，请稍后重试"})
            
    except Exception as e:
        print(f"❌ 语音生成异常: {e}")
        send_feishu_message(chat_id, "text", {"text": "❌ 语音生成异常，请稍后重试"})

print("✅ 飞书事件处理路由定义完成")


In [ ]:
# 启动API服务器和ngrok隧道
from pyngrok import ngrok
import time

# 定义API服务器运行函数
def run_api_server():
    """在独立线程中运行FastAPI服务器"""
    try:
        # 使用uvicorn运行FastAPI应用
        # host="0.0.0.0": 监听所有网络接口
        # port=7860: 使用端口7860（Hugging Face Spaces常用端口）
        # log_level="info": 设置日志级别
        uvicorn.run(
            app,
            host="0.0.0.0",
            port=7860,
            log_level="info"
        )
    except Exception as e:
        print(f"❌ API服务器启动失败: {e}")

# 在后台线程启动API服务器
print("🚀 正在启动FastAPI服务器...")
print("📡 服务器地址: http://localhost:7860")
print("⏱️  服务器启动需要几秒钟时间")

# 创建并启动服务器线程
server_thread = threading.Thread(target=run_api_server)
server_thread.daemon = True  # 设置为守护线程
server_thread.start()

# 等待服务器启动
time.sleep(5)

try:
    print("🌐 正在创建ngrok隧道...")
    
    # 创建HTTP隧道到本地7860端口
    public_tunnel = ngrok.connect(7860)
    public_url = public_tunnel.public_url
    
    print("\n✅ 系统启动成功！")
    print("="*60)
    print(f"🌍 公网访问地址: {public_url}")
    print(f"📋 API文档地址: {public_url}/docs")
    print(f"❤️  健康检查: {public_url}/health")
    print(f"🎵 TTS端点: {public_url}/tts")
    print(f"🤖 飞书事件回调: {public_url}/feishu/events")
    print("="*60)
    
    # 保存公网URL供后续使用
    PUBLIC_API_URL = public_url
    
    print("\n📋 下一步配置：")
    print("1. 复制飞书事件回调URL：")
    print(f"   {public_url}/feishu/events")
    print("2. 在飞书开放平台配置事件订阅")
    print("3. 测试机器人功能")
    
except Exception as e:
    print(f"❌ ngrok隧道创建失败: {e}")
    print("🔧 可能的解决方案：")
    print("   1. 检查ngrok token是否正确")
    print("   2. 确认FastAPI服务器已启动")
    print("   3. 重启Colab运行时")
    PUBLIC_API_URL = None


In [ ]:
# 生成飞书机器人配置指南
print("🤖 飞书机器人详细配置指南")
print("="*60)
print()

print("📋 第一步：权限配置")
print("在飞书开放平台 -> 权限管理页面，开启以下权限：")
print()
print("🔹 **消息与群组权限**：")
print("   • 获取与发送单聊、群组消息")
print("   • 读取用户发给机器人的单聊消息")
print("   • 获取群组基本信息")
print("   • 发送文件消息")
print()
print("🔹 **通讯录权限**：")
print("   • 获取用户基本信息")
print("   • 获取用户邮箱信息（可选）")
print()

print("📋 第二步：事件订阅配置")
print("在飞书开放平台 -> 事件订阅页面：")
print()
if 'PUBLIC_API_URL' in locals() and PUBLIC_API_URL:
    print(f"🔗 **请求网址**：{PUBLIC_API_URL}/feishu/events")
else:
    print("🔗 **请求网址**：[您的API地址]/feishu/events")
    
print("🔑 **Verification Token**：复制Token并填入上面的代码中")
print("🔐 **Encrypt Key**：如果启用加密，复制密钥")
print()
print("🔹 **订阅事件**（必须订阅）：")
print("   • 接收消息 - im.message.receive_v1")
print("   • 消息已读 - im.message.message_read_v1（可选）")
print()

print("📋 第三步：机器人配置")
print("在飞书开放平台 -> 机器人页面：")
print("   • 启用机器人功能")
print("   • 设置机器人名称：Turkish TTS Bot")
print("   • 设置机器人描述：土耳其语文本转语音助手")
print("   • 上传机器人头像（可选）")
print()

print("📋 第四步：发布和安装")
print("1. 在应用详情页点击\"创建版本\"")
print("2. 填写版本信息并提交审核")
print("3. 审核通过后点击\"发布\"")
print("4. 在\"应用发布\"页面添加可用范围（用户或部门）")
print("5. 用户可以在飞书中搜索并添加机器人")
print()

print("📋 第五步：测试机器人")
print("1. 在飞书中搜索机器人名称")
print("2. 添加机器人为联系人")
print("3. 发送 /help 查看使用说明")
print("4. 发送 /test 测试TTS功能")
print("5. 发送土耳其语文本测试语音转换")
print()

print("💡 常见问题：")
print("• 如果收不到消息，检查事件订阅配置")
print("• 如果权限错误，确认已开启必要权限")
print("• 如果API调用失败，检查App ID和Secret")
print("• 如果文件上传失败，确认已开启文件发送权限")

print("\n🎯 配置检查清单：")
print("□ App ID 和 App Secret 已配置")
print("□ Verification Token 已配置")
print("□ 事件订阅 URL 已设置")
print("□ 消息权限已开启")
print("□ 文件发送权限已开启")
print("□ 机器人功能已启用")
print("□ 应用已发布并添加用户")

# 配置验证
config_ok = True
if FEISHU_APP_ID == "YOUR_FEISHU_APP_ID":
    print("\n❌ 请配置 FEISHU_APP_ID")
    config_ok = False
if FEISHU_APP_SECRET == "YOUR_FEISHU_APP_SECRET":
    print("❌ 请配置 FEISHU_APP_SECRET")
    config_ok = False
if FEISHU_VERIFICATION_TOKEN == "YOUR_VERIFICATION_TOKEN":
    print("❌ 请配置 FEISHU_VERIFICATION_TOKEN")
    config_ok = False

if config_ok:
    print("\n✅ 飞书配置信息已设置完成")
else:
    print("\n⚠️  请先完成飞书配置，然后重新运行相关代码单元")


In [ ]:
# 测试TTS功能和飞书API
import IPython.display as ipd
import requests

print("🧪 系统功能测试")
print("="*40)

# 测试1：TTS模型功能
print("\n📋 测试1：TTS模型功能")
test_text = "Merhaba! Bu bir test mesajıdır. Türkçe metin okuma testi."
print(f"🧪 测试文本: {test_text}")

try:
    # 生成测试语音
    print("🎵 正在生成测试语音...")
    
    if 'SPEAKER_WAV_PATH' in locals() and SPEAKER_WAV_PATH and os.path.exists(SPEAKER_WAV_PATH):
        # 使用声音克隆
        tts.tts_to_file(
            text=test_text,
            file_path="test_output.wav",
            speaker_wav=SPEAKER_WAV_PATH,
            language="tr"  # 土耳其语
        )
        print("✅ 测试语音生成成功（使用声音克隆）")
    else:
        # 使用默认说话人
        tts.tts_to_file(
            text=test_text,
            file_path="test_output.wav",
            language="tr"  # 土耳其语
        )
        print("✅ 测试语音生成成功（使用默认说话人）")
    
    # 播放生成的语音
    print("🎧 生成的语音预览：")
    display(ipd.Audio("test_output.wav"))
    
except Exception as e:
    print(f"❌ TTS测试失败: {e}")

# 测试2：飞书API连接
print("\n📋 测试2：飞书API连接")
try:
    access_token = get_feishu_access_token()
    if access_token:
        print("✅ 飞书访问令牌获取成功")
        print(f"🔑 Token: {access_token[:20]}...")
    else:
        print("❌ 飞书访问令牌获取失败")
        print("💡 请检查App ID和App Secret配置")
except Exception as e:
    print(f"❌ 飞书API测试失败: {e}")

# 测试3：API接口
if 'PUBLIC_API_URL' in locals() and PUBLIC_API_URL:
    print("\n📋 测试3：API接口")
    
    # 健康检查
    try:
        print("🌐 测试健康检查接口...")
        response = requests.get(f"{PUBLIC_API_URL}/health", timeout=10)
        
        if response.status_code == 200:
            health_data = response.json()
            print("✅ 健康检查通过")
            print(f"📊 TTS模型: {health_data.get('tts_model')}")
            print(f"🎤 音频样本: {health_data.get('speaker_sample')}")
            print(f"🤖 飞书API: {health_data.get('feishu_api')}")
        else:
            print(f"⚠️  健康检查异常，状态码: {response.status_code}")
    except Exception as e:
        print(f"❌ 健康检查失败: {e}")
    
    # TTS API测试
    print("\n🌐 测试TTS API接口...")
    test_payload = {
        "text": "API test mesajı. Feishu entegrasyonu çalışıyor mu?",
        "language": "tr",
        "user_id": "test_user"
    }
    
    try:
        print(f"📤 发送请求: {test_payload['text']}")
        
        response = requests.post(
            f"{PUBLIC_API_URL}/tts",
            json=test_payload,
            timeout=60  # 增加超时时间
        )
        
        if response.status_code == 200:
            # 保存返回的MP3文件
            api_test_filename = "feishu_api_test_output.mp3"
            with open(api_test_filename, "wb") as f:
                f.write(response.content)
            
            print("✅ TTS API测试成功！")
            print(f"📁 生成文件: {api_test_filename}")
            print(f"📊 文件大小: {len(response.content) / 1024:.1f} KB")
            
            # 播放生成的音频
            print("🎧 API生成的语音：")
            display(ipd.Audio(api_test_filename))
            
        else:
            print(f"❌ TTS API测试失败，状态码: {response.status_code}")
            print(f"📄 错误信息: {response.text}")
            
    except requests.exceptions.Timeout:
        print("⏰ API请求超时（这是正常的，TTS生成需要时间）")
        print("💡 请稍后手动测试API")
    except Exception as api_error:
        print(f"❌ TTS API测试异常: {api_error}")
        
else:
    print("\n❌ 无法测试API，因为公网URL未生成")

# 测试4：文件上传功能
print("\n📋 测试4：文件上传功能")
if os.path.exists("test_output.wav"):
    try:
        # 转换为MP3用于测试上传
        sound = AudioSegment.from_wav("test_output.wav")
        sound.export("upload_test.mp3", format="mp3")
        
        print("🌐 测试飞书文件上传...")
        file_key = upload_feishu_file("upload_test.mp3")
        
        if file_key:
            print(f"✅ 文件上传成功，file_key: {file_key}")
        else:
            print("❌ 文件上传失败")
            print("💡 请检查飞书API配置和文件发送权限")
            
        # 清理测试文件
        if os.path.exists("upload_test.mp3"):
            os.remove("upload_test.mp3")
            
    except Exception as e:
        print(f"❌ 文件上传测试失败: {e}")
else:
    print("⚠️  跳过文件上传测试（缺少测试音频文件）")

print("\n🎯 测试总结：")
print("✅ 所有测试完成")
print("💡 如果某些测试失败，请检查相应的配置")
print("🚀 如果所有测试通过，可以开始使用飞书机器人！")
